In [590]:

import numpy as np
from collections import Counter

In [591]:
data = [
    [12.0, 1.5, 1, 'Wine'],
    [5.0, 2.0, 0, 'Beer'],
    [40.0, 0.0, 1, 'Whiskey'],
    [13.5, 1.2, 1, 'Wine'],
    [4.5, 1.8, 0, 'Beer'],
    [38.0, 0.1, 1, 'Whiskey'],
    [11.5, 1.7, 1, 'Wine'],
    [5.5, 2.3, 0, 'Beer']
]
data = np.array(data)
data

array([['12.0', '1.5', '1', 'Wine'],
       ['5.0', '2.0', '0', 'Beer'],
       ['40.0', '0.0', '1', 'Whiskey'],
       ['13.5', '1.2', '1', 'Wine'],
       ['4.5', '1.8', '0', 'Beer'],
       ['38.0', '0.1', '1', 'Whiskey'],
       ['11.5', '1.7', '1', 'Wine'],
       ['5.5', '2.3', '0', 'Beer']], dtype='<U32')

In [592]:
def data_split(data, n_features):
    X = data[:, 0:n_features]
    X = X.astype(float)
    y = data[:, n_features]
    return X, y

In [593]:
n_features = 3
X, y = data_split(data, n_features)
labels = ["Wine", "Beer", "Whiskey"]

### Gini Impurity
  Gini Impurity is one of the commonly used criterion to evalutate the goodness of a split, the lower the gini impurity, the better the split. Mathematically, it is defined as
  
  $$
    G(S) = 1 - \sum_{i=1}^n p_i^{2}  
  $$
  
  Where $p_i$ is the proportion of elements in the $i^{th}$ class.
  $$
      0 \leq G(S) \leq 1
  $$
  We get 0 for a pure class, consisting of only one value and 1 for maximum impurity

In [594]:
def gini_impurity(y):
    #creating a frequency list for the labels
    labels_list = [0, 0, 0]
    for i in range(len(y)):
        for j in range(len(labels)):
            if y[i] == labels[j]:
                labels_list[j] += 1
    gini = 1
    #calculating gini value
    for i in range(len(labels_list)):
        if(sum(labels_list) == 0):
            return 1
        gini -= (labels_list[i]/sum(labels_list))**2
    return gini

gini_impurity(y)
            

0.65625

We must return the threshold with the least **weighted gini-impurity** which is calculated for a column as 
$$
    G_{split} = \frac{N_L}{N} G_L + \frac{N_R}{N}G_R
$$
Where $G_L$ and $G_R$ are the gini impurities for the left and right sides of the threshold respectively

In [595]:

    def weighted_gini(left_y, right_y):
        total = len(left_y) + len(right_y)
        if total == 0:
            return 0
        gini_left = gini_impurity(left_y)
        gini_right = gini_impurity(right_y)
        return (len(left_y) * gini_left + len(right_y) * gini_right) / total

### Calculation Of Entropy 
Entropy is alternative function used to compute like gini used in decision trees. It is similarly used to find the best split, it is calculated as

$$
    H(S) = -\sum_{i = 1}^{c}p_i \log_2 p_i
$$

$p_i$ simply represents the proportion of elements in the $i^{th}$ class

In [596]:
def entropy(y):
    labels = ["Wine", "Beer", "Whiskey"]
    labels_list = [0, 0, 0]
    for i in range(len(y)):
        for j in range(len(labels)):
            if y[i] == labels[j]:
                labels_list[j] += 1  # Increment count for the corresponding label

    entropy_value = 0
    for i in range(len(labels_list)):
        p = labels_list[i] / len(y)  # Probability of each label
        if p > 0:  # To prevent log(0), which is undefined
            entropy_value -= p * np.log2(p)
    
    return entropy_value


### Information Gain

**Information Gain** uses entropy to measure the effectiveness of a split in a dataset. If we split a dataset \( S \) using a feature \( A \), the Information Gain is calculated as:

$$
IG(S, A) = H(S) - \sum_{v \in \text{values}(A)} \frac{|S_v|}{|S|} H(S_v)
$$

Where:

- $H(S)$ is the entropy of the original dataset $ S$  
- $\text{values}(A)$ are the unique values of feature $A$  
- $S_v$ is the subset of $S$ where $A = v$  

The feature and threshold with the **highest Information Gain** is selected for the split.

It can also be written as:

$$
\text{Information Gain} = \text{Entropy before split} - \text{Entropy after split}
$$


In [597]:
def information_gain(X_column, y, threshold):
    left_mask = X_column <= threshold
    right_mask = X_column > threshold

    y_left = y[left_mask]
    y_right = y[right_mask]

    entropy_before = entropy(y)
    entropy_after = (len(y_left) / len(y)) * entropy(y_left) + (len(y_right) / len(y)) * entropy(y_right)

    gain = entropy_before - entropy_after
    return gain


###  Method: Node Initialization

Initializes a node in the decision tree.

**Parameters:**

- `feature_index`: Index of the feature to split on
- `threshold`: Threshold value for the split
- `left`: Left child node
- `right`: Right child node
- `value`: Predicted class if the node is a leaf
- `depth`: Depth of the node in the tree
- `gini`: Gini impurity of the node


In [598]:


class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None, depth=None, gini=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        self.depth = depth
        self.gini = gini

    @staticmethod
    def split(X, y, feature_index, threshold):
        left_mask = X[:, feature_index] <= threshold
        right_mask = X[:, feature_index] > threshold

        left_X = X[left_mask]
        left_y = y[left_mask]
        right_X = X[right_mask]
        right_y = y[right_mask]

        return left_X, right_X, left_y, right_y

    @staticmethod
    def build_tree(X, y, depth, param):
        max_depth = 10

        # Base case
        if depth >= max_depth or len(np.unique(y)) == 1:
            leaf_value = Counter(y).most_common(1)[0][0]
            return Node(value=leaf_value, depth=depth, gini=0)


        # Try finding best split
        best_split = best_split_fn(X, y, param) 
        if best_split is None:
            leaf_value = Counter(y).most_common(1)[0][0]
            return Node(value=leaf_value, depth=depth, gini=0)

        left_X, right_X, left_y, right_y, best_feature, best_threshold = best_split

        left_node = Node.build_tree(left_X, left_y, depth + 1, param)
        right_node = Node.build_tree(right_X, right_y, depth + 1, param)

        return Node(
            feature_index=best_feature,
            threshold=best_threshold,
            left=left_node,
            right=right_node,
            gini=None,  # or calculate if needed
            depth=depth
        )


For obtaining the candidate thresholds at each feature, we first sort the array and the find the midpoints of each successive 2 elements

In [599]:
    def get_thresholds(feature_arr):
        feature_arr = np.sort(np.unique(feature_arr))
        thresholds = []
        for i in range(len(feature_arr) - 1):
            thresholds.append((feature_arr[i] + feature_arr[i+1])/2)
        return thresholds
    
    Node.get_thresholds = staticmethod(get_thresholds)

Function to determine the best split at each feature index based on the **information gain** or **gini-impurity**

In [600]:
def best_split_fn(X, y, param):
    if param == "entropy":
        #initializing best_gain to 0, since we must maximize information gain
        best_gain = 0.0
        best_threshold = None
        best_feature = None
        best_split = None
        
        #loop through each feature in the feature matrix
        for feature_index in range(X.shape[1]):
            thresholds = Node.get_thresholds(X[:, feature_index])
            
            #evaluate information gain at each possible threshold, and returning the optimal threshold value
            for threshold in thresholds:
                left_X, right_X, left_y, right_y = Node.split(X, y, feature_index, threshold)
                if len(left_y) == 0 or len(right_y) == 0:
                    continue
                
                gain = information_gain(X[:, feature_index], y, threshold)
                if gain >= best_gain:
                    best_gain = gain
                    best_feature = feature_index
                    best_threshold = threshold
                    best_split = (left_X, right_X, left_y, right_y, best_feature, best_threshold)
        
        #returning best feature and threshold
        return best_split
    
    #similarly for the gini part, just copied the entropy function
    elif param == "gini":
            best_gini = 1.01
            best_threshold = None
            best_feature = None
            best_split = None
            for feature_index in range(X.shape[1]):
                thresholds = Node.get_thresholds(X[:, feature_index])
                for threshold in thresholds:
                    left_X, right_X, left_y, right_y = Node.split(X, y, feature_index, threshold)
                    if len(left_y) == 0 or len(right_y) == 0:
                        continue
                    gini = weighted_gini(left_y, right_y)
                    if gini <= best_gini:
                        best_gini = gini
                        best_feature = feature_index
                        best_threshold = threshold
                        best_split = (left_X, right_X, left_y, right_y, best_feature, best_threshold)
            return best_split


In [601]:
    def predict_one(self, x):
        # If there are no children
        if self.value is not None:
            return self.value

        # if value is less than threshold, we go to left child
        if x[self.feature_index] <= self.threshold:
            return self.left.predict_one(x)
        #if value is greater than threshold, we go to right child
        else:
            return self.right.predict_one(x)
        
Node.predict_one = predict_one

In [602]:
def predict(self, X):
        return np.array([self.predict_one(x) for x in X])
Node.predict = predict

In [603]:
#function to predict labels for test data using parameter as gini
def evaluation_gini():
    tree = Node.build_tree(X, y, depth = 0, param = "gini")
    test_data = np.array([
    [6.0, 2.1, 0],   # Expected: Beer
    [39.0, 0.05, 1], # Expected: Whiskey
    [13.0, 1.3, 1]   # Expected: Wine
    ])
    print(tree.predict(test_data))

#function to plot the tree
def plot_tree_gini():
    tree = Node.build_tree(X, y,depth = 0, param = "gini")
    test_data = np.array([
    [6.0, 2.1, 0],   # Expected: Beer
    [39.0, 0.05, 1], # Expected: Whiskey
    [13.0, 1.3, 1]   # Expected: Wine
    ])
    feature_names = ["Alcohol_Content", "Sugar", "Color"]
    print_tree(tree, feature_names, max_depth = np.inf)

In [606]:
#function to predict labels for test data using parameter as information gain
def evaluation_entropy():
    tree = Node.build_tree(X, y, depth = 0, param = "entropy")
    test_data = np.array([
    [6.0, 2.1, 0],   # Expected: Beer
    [39.0, 0.05, 1], # Expected: Whiskey
    [13.0, 1.3, 1]   # Expected: Wine
    ])
    print(tree.predict(test_data))

#function to plot the tree
def plot_tree_entropy():
    tree = Node.build_tree(X, y, depth = 0, param = "entropy")
    test_data = np.array([
    [6.0, 2.1, 0],   # Expected: Beer
    [39.0, 0.05, 1], # Expected: Whiskey
    [13.0, 1.3, 1]   # Expected: Wine
    ])
    feature_names = ["Alcohol_Content", "Sugar", "Color"]
    print_tree(tree, feature_names, max_depth = np.inf)

In [604]:
evaluation_gini()

['Beer' 'Whiskey' 'Wine']


In [605]:
plot_tree_gini()

if Color <= 0.50:
    Predict -> Beer
else:
    if Sugar <= 0.65:
        Predict -> Whiskey
    else:
        Predict -> Wine


In [607]:
evaluation_entropy()

['Beer' 'Whiskey' 'Wine']


In [608]:
plot_tree_entropy()

if Color <= 0.50:
    Predict -> Beer
else:
    if Sugar <= 0.65:
        Predict -> Whiskey
    else:
        Predict -> Wine
